In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

### 1. Prepare Data

In [2]:
torch.manual_seed(777)  # reproducibility

idx2char = ['h', 'i', 'e', 'l', 'o']

# Teach hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
y_data = [1, 0, 2, 3, 3, 4]    # ihello

# As we have one batch of samples, we will change them to variables only once
inputs = Variable(torch.LongTensor(x_data))
labels = Variable(torch.LongTensor(y_data))

### 2. Set Parameters

In [3]:
num_classes = 5
input_size = 5
embedding_size = 10  # embedding size
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6
num_layers = 1  # one-layer rnn

### 3. Define Model

In [15]:
class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        
        self.embedding = nn.Embedding(input_size, embedding_size) #Embedding(vocab_size, embedding_size)
        self.rnn = nn.RNN(input_size=embedding_size, hidden_size=5, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size)
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))

        emb = self.embedding(x)
        emb = emb.view(batch_size, sequence_length, -1)

        # Propagate embedding through RNN
        # Input: (batch, seq_len, embedding_size)
        # h_0: (num_layers * num_directions, batch, hidden_size)
        out, _ = self.rnn(emb, h_0)
        
        return self.fc(out.view(-1, num_classes))


# Instantiate RNN model
model = Model()
print(model)

Model(
  (embedding): Embedding(5, 10)
  (rnn): RNN(10, 5, batch_first=True)
  (fc): Linear(in_features=5, out_features=5, bias=True)
)


### 4. Construct Loss function and Optimizer

In [16]:
# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [17]:
# Train the model
for epoch in range(100):
    outputs = model(inputs)
    optimizer.zero_grad()
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    result_str = [idx2char[c] for c in idx.squeeze()]
    print("epoch: %d, loss: %1.3f" % (epoch + 1, loss.data))
    print("Predicted string: ", ''.join(result_str))

print("Learning finished!")

epoch: 1, loss: 1.596
Predicted string:  eleloo
epoch: 2, loss: 1.274
Predicted string:  elelll
epoch: 3, loss: 1.061
Predicted string:  elelll
epoch: 4, loss: 0.912
Predicted string:  elelll
epoch: 5, loss: 0.788
Predicted string:  elelll
epoch: 6, loss: 0.690
Predicted string:  ihilll
epoch: 7, loss: 0.589
Predicted string:  ihilll
epoch: 8, loss: 0.487
Predicted string:  ihillo
epoch: 9, loss: 0.423
Predicted string:  ihello
epoch: 10, loss: 0.395
Predicted string:  ihello
epoch: 11, loss: 0.374
Predicted string:  ihello
epoch: 12, loss: 0.287
Predicted string:  ihello
epoch: 13, loss: 0.280
Predicted string:  ihello
epoch: 14, loss: 0.213
Predicted string:  ihello
epoch: 15, loss: 0.199
Predicted string:  ihello
epoch: 16, loss: 0.174
Predicted string:  ihello
epoch: 17, loss: 0.137
Predicted string:  ihello
epoch: 18, loss: 0.117
Predicted string:  ihello
epoch: 19, loss: 0.109
Predicted string:  ihello
epoch: 20, loss: 0.083
Predicted string:  ihello
epoch: 21, loss: 0.070
Predic